### faiss 적용

In [17]:
import cv2
import numpy as np
import faiss

def compute_vlad(image, des, labels, centers):
    sift = cv2.SIFT_create(edgeThreshold=80)
    _, des = sift.detectAndCompute(image, None)
    
    if des is None: 
        return None
    
    vlad = np.zeros((centers.shape[0], des.shape[1]), dtype=np.float32)
    
    for i in range(des.shape[0]):
        vlad[labels[i]] += des[i] - centers[labels[i]]
        
    vlad = cv2.normalize(vlad, None).flatten()
    vlad /= np.linalg.norm(vlad)
    return vlad

def similar_images(query_image, category_images, k):
    sift = cv2.SIFT_create(edgeThreshold=80)
    
    query_image = cv2.resize(query_image, (300, 300))
    _, query_des = sift.detectAndCompute(query_image, None)
    if query_des is None:
        return None

    category_des = []
    for category, images in category_images.items():
        for image in images:
            _, des = sift.detectAndCompute(image, None)
            if des is not None:
                category_des.extend(des)

    category_des = np.array(category_des)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, labels, centers = cv2.kmeans(category_des, k, None, criteria, 5, flags)

    query_vlad = compute_vlad(query_image, query_des, labels, centers)
    if query_vlad is None:
        return None

    similarity_scores = {}
    for category, images in category_images.items():
        category_vlads = []
        for image in images:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (300, 300))
            _, des = sift.detectAndCompute(image, None)
            if des is not None:
                category_vlads.append(compute_vlad(image, des, labels, centers))

        category_vlads = np.array(category_vlads)

        # Faiss Index 생성
        d = query_vlad.shape[0] # 벡터 정보 불러오기
        index = faiss.IndexFlatL2(d)
        index.add(category_vlads)

        # Query 이미지를 Faiss Index에 검색
        distances, _ = index.search(query_vlad.reshape(1, -1), 5)
        similarity_scores[category] = np.mean(distances)

    sorted_categories = sorted(similarity_scores.items(), key=lambda x: x[1])
    return sorted_categories

path = 'c:/data/temp/'

# query_image_path =  path + 'flower6.jpg'
# query_image_path =  path + 'mouse6.jpg'
query_image_path =  path + 'umb6.jpg'

categories = {
    'mouse': [cv2.imread(path + 'mouse1.jpg'), cv2.imread(path + 'mouse2.jpg'), cv2.imread(path + 'mouse3.jpg'), 
              cv2.imread(path + 'mouse4.jpg'), cv2.imread(path + 'mouse5.jpg')],
    'flower': [cv2.imread(path + 'flower1.jpg'), cv2.imread(path + 'flower2.jpg'), cv2.imread(path + 'flower3.jpg'), 
               cv2.imread(path + 'flower4.jpg'), cv2.imread(path + 'flower5.jpg')],
    'umbrella': [cv2.imread(path + 'umb1.jpg'), cv2.imread(path + 'umb2.jpg'), cv2.imread(path + 'umb3.jpg'),
                 cv2.imread(path + 'umb4.jpg'), cv2.imread(path + 'umb5.jpg')]
}

query_image = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)

k = 2
categories = similar_images(query_image, categories, k)

print('0에 가까울 수록 입력 이미지와 유사하다.')
for k, v in categories:
    print("카테고리: %-10s / 유사도: %1.2f"% (k , v))

0에 가까울 수록 입력 이미지와 유사하다.
카테고리: umbrella   / 유사도: 0.11
카테고리: mouse      / 유사도: 0.21
카테고리: flower     / 유사도: 0.96
